### 1. 每月一次，查找最近月K线MACD指标在零下金叉，且现在已经跃上零线，并现在尚未死叉

In [ ]:
import yfinance as yf
import pandas as pd


def calculate_ema(data, span):
    """计算EMA"""
    return data.ewm(span=span, adjust=False).mean()


def calculate_macd(df, short=12, long=26, signal=9):
    """计算MACD指标"""
    df["EMA_short"] = calculate_ema(df["Close"], short)
    df["EMA_long"] = calculate_ema(df["Close"], long)
    df["DIF"] = df["EMA_short"] - df["EMA_long"]
    df["DEA"] = calculate_ema(df["DIF"], signal)
    df["MACD_hist"] = 2 * (df["DIF"] - df["DEA"])
    return df


def find_macd_golden_cross(df):
    """查找MACD金叉的日期"""
    df["Golden_Cross"] = (df["DIF"] > df["DEA"]) & (
        df["DIF"].shift(1) <= df["DEA"].shift(1)
    )
    golden_cross_dates = df[df["Golden_Cross"]].index
    return golden_cross_dates


# 初始化
stock_code = "601669"
data = yf.download(stock_code, start="2020-01-01", end="2020-12-12")
data.columns = data.columns.droplevel(1)  # 去掉第二层的ticker，变为单层索引

# 转换为月线数据，取每月最后一个交易日的收盘价
monthly_data = (
    data.resample("M")
    .agg({"Open": "first", "High": "max", "Low": "min", "Close": "last"})
    .dropna()
)

# 计算月K线的MACD指标
monthly_data = calculate_macd(monthly_data)

# 查找MACD金叉
golden_cross_dates = find_macd_golden_cross(monthly_data)

# 输出金叉的日期
print("月线MACD金叉日期:")
golden_cross_dates

Failed to get ticker '601669' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['601669']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


月线MACD金叉日期:


DatetimeIndex([], dtype='datetime64[ns]', name='Date', freq='M')

: 